# Psycellium: A Decentralized Cooperatives Network

Psycellium is a blockchain network for cooperatives with the goal of bridging the gap between local communities and foster financial growth among its members. Services include loans, crowdfunding, and investments that can be done between registered cooperatives regardless of where they are. This opens a path for globalization and maximizing the utilization of blockchain in solving inter-border problems in transactions between communities.

Draft Website (Not Complete): https://psycellium.herokuapp.com <br>
Github Link: https://github.com/kyleaquino/psycellium

Note: Due to other important concerns I'm currently handling. I just developed a Minimum Viable Product Solution that introduces the fundamentals of how the platform should work.

# The Code

- Used python as my main programming language that will interact with the smart contract. 
- Used the Ethereum Libraries available for Python (Web3.py, solc) 
- Deployed the contract into the Ropsten Test Network

In [1]:
import os
from web3 import Web3
from eth_account import Account
from solc import compile_files, compile_source
from web3.contract import ConciseContract

This initialize the Ethereum node that will be used to connect with the ropsten etheruem test network via infuria

In [2]:
def initWeb3():
    web3=Web3(Web3.HTTPProvider('https://ropsten.infura.io/v3/6c25a8476cd941aab5961e19adde9502'))
    print("Web3 is Conntected:", web3.isConnected())
    return web3

This will be the general function for the creation of new ethereum wallets

In [3]:
def createAccount(password):
    account = Account.create(password)
    return account, password

This creates a new cooperative in the ethereum network, each contract represents a single cooperative, the contract address represents their id to verify the members and transactions facilitated inside the organization. The sender/deployer of the contract will be temporarily be the director of the cooperative to facilitate the transactions

In [55]:
def createCoop(web3, name, description, account):
    # Deploy New Contract
    compiled_sol=compile_files(['contracts/Cooperatives.sol'])
    interface=compiled_sol['contracts/Cooperatives.sol:Cooperatives']
    contract=web3.eth.contract(abi=interface['abi'], bytecode=interface['bin'])
    
    # Prepare Transaction 
    txn = {
        'chainId': 3,
        'gas': contract.constructor(name, description).estimateGas()*2,
        'gasPrice': web3.toWei('1', 'gwei'),
        'nonce': web3.eth.getTransactionCount(account.address)
    }
    
    # Deploy Transaction
    deploy_txn=contract.constructor(name, description).buildTransaction(txn)
    signed=web3.eth.account.signTransaction(deploy_txn, account.privateKey)
    txn_hash=web3.eth.sendRawTransaction(signed.rawTransaction)  
    txn_receipt=web3.eth.waitForTransactionReceipt(txn_hash)
    contract = web3.eth.contract(address=txn_receipt.contractAddress,abi=contract.abi)
    
    print("HASH:", txn_hash)
    print("RECIEPT:", txn_receipt)
    return contract

Anyone can join the cooperative deployed in the ethereum network the default registration fee is set to 200000000000000000 wei (around 25 USD) 
*Will add a functionality to change the fee requirement

In [69]:
def joinCoop(web3, contract, sender, amount):
    txn = {
        'chainId': 3,
        'value': amount,
        'gasPrice': web3.toWei('1', 'gwei'),
        'nonce': web3.eth.getTransactionCount(sender.address)
    }
    txn['gas'] = 1000000
    print(txn['gas'])
    deploy_txn=contract.functions.joinCoop(amount).buildTransaction(txn)
    signed=web3.eth.account.signTransaction(deploy_txn, sender.privateKey)
    txn_hash=web3.eth.sendRawTransaction(signed.rawTransaction)  
    txn_receipt=web3.eth.waitForTransactionReceipt(txn_hash)
    print("HASH:", txn_hash)
    print("RECIEPT:", txn_receipt)

This sets new directors in the cooperatives. The candidate must first be a cooperatives member to be eligible to become a director

In [83]:
def setDirector(web3, contract, sender, member):
    txn = {
        'chainId': 3,
        'gas': contract.functions.setDirector(member.address).estimateGas()*2,
        'gasPrice': web3.toWei('1', 'gwei'),
        'nonce': web3.eth.getTransactionCount(sender.address)
    } 
    deploy_txn=contract.functions.setDirector(member.address).buildTransaction(txn)
    signed=web3.eth.account.signTransaction(deploy_txn, sender.privateKey)
    txn_hash=web3.eth.sendRawTransaction(signed.rawTransaction)  
    txn_receipt=web3.eth.waitForTransactionReceipt(txn_hash)
    print("HASH:", txn_hash)
    print("RECIEPT:", txn_receipt)

Each member can deposit their funds into their cooperatives. The deposited amount will be stored into the contract address of the cooperatives and is recorded through the event logs. the benefits of choosing the cooperatives as their bank will be done offchain

In [21]:
def deposit(web3, contract, sender, amount):
    txn = {
        'to': contract.address,
        'from': sender.address,
        'chainId': 3,
        'value': amount,
        'gas': contract.functions.deposit(amount).estimateGas()*2,
        'gasPrice': web3.toWei('1', 'gwei'),
        'nonce': web3.eth.getTransactionCount(sender.address)
    }
    deploy_txn=contract.functions.deposit(amount).buildTransaction(txn)
    signed=web3.eth.account.signTransaction(deploy_txn, sender.privateKey)
    txn_hash=web3.eth.sendRawTransaction(signed.rawTransaction)  
    txn_receipt=web3.eth.waitForTransactionReceipt(txn_hash)
    print("HASH:", txn_hash)
    print("RECIEPT:", txn_receipt)

Each member is also free from withdrawing the amount they deposited

In [22]:
def withdraw(web3, contract, sender, amount):
    txn = {
        'chainId': 3,
        'gas': contract.functions.withdraw(amount).estimateGas()*2,
        'gasPrice': web3.toWei('1', 'gwei'),
        'nonce': web3.eth.getTransactionCount(sender.address)
    }
    deploy_txn=contract.functions.withdraw(amount).buildTransaction(txn)
    signed=web3.eth.account.signTransaction(deploy_txn, sender.privateKey)
    txn_hash=web3.eth.sendRawTransaction(signed.rawTransaction)  
    txn_receipt=web3.eth.waitForTransactionReceipt(txn_hash)
    print("HASH:", txn_hash)
    print("RECIEPT:", txn_receipt)

The cooperatives can transfer funds to another address by the approval of the directors

In [116]:
def submitRequest(web3, contract, sender, destination, amount):
    txn = {
        'chainId': 3, 
        'gasPrice': web3.toWei('1', 'gwei'),
        'nonce': web3.eth.getTransactionCount(sender.address)
    }
    txn['gas'] = contract.functions.sumbitRequest(destination.address, amount, bytes(0)).estimateGas() #Fix 'Sumbit' Typo
    deploy_txn=contract.functions.sumbitRequest(destination.address, amount, bytes(0)).buildTransaction(txn) #Fix 'Sumbit' Typo
    signed=web3.eth.account.signTransaction(deploy_txn, sender.privateKey)
    txn_hash=web3.eth.sendRawTransaction(signed.rawTransaction)  
    txn_receipt=web3.eth.waitForTransactionReceipt(txn_hash)
    print("HASH:", txn_hash)
    print("RECIEPT:", txn_receipt)

Only the directors can confirm the transaction. Once the confirmation is complete the funds are transfered to the destination address

In [24]:
def confirmTransaction(web3, contract, sender, txn_id):
    txn = {
        'chainId': 3, 
        'gas': contract.functions.confirmTransaction(txn_id).estimateGas()*2,
        'gasPrice': web3.toWei('1', 'gwei'),
        'nonce': web3.eth.getTransactionCount(sender.address)
    }
    deploy_txn=contract.functions.confirmTransaction(txn_id).buildTransaction(txn)
    signed=web3.eth.account.signTransaction(deploy_txn, sender.privateKey)
    txn_hash=web3.eth.sendRawTransaction(signed.rawTransaction)  
    txn_receipt=web3.eth.waitForTransactionReceipt(txn_hash)
    print("HASH:", txn_hash)
    print("RECIEPT:", txn_receipt)

Only the directors can revoke the transaction. This revokes the pending request

In [25]:
def revokeTransaction(web3, contract, sender, txn_id):
    txn = {
        'chainId': 3, 
        'gas': contract.functions.revokeTransaction(txn_id).estimateGas()*2,
        'gasPrice': web3.toWei('1', 'gwei'),
        'nonce': web3.eth.getTransactionCount(sender.address)
    }
    deploy_txn=contract.functions.revokeTransaction(txn_id).buildTransaction(txn)
    signed=web3.eth.account.signTransaction(deploy_txn, sender.privateKey)
    txn_hash=web3.eth.sendRawTransaction(signed.rawTransaction)  
    txn_receipt=web3.eth.waitForTransactionReceipt(txn_hash)
    print("HASH:", txn_hash)
    print("RECIEPT:", txn_receipt)

# Demo

This demo presents how the contract work and how it will integrate to the user interface

1. I created first the deployer of the contract which will be the temporary director of the cooperative

In [12]:
web3=initWeb3()
account, password = createAccount('admin1234')
web3.eth.defaultAccount = account.address

print("Account:", account.address)
print("Private Key:", account.privateKey)

True
Account: 0xF3e2DB0F694EC09D5546298fa7B801D37A3b45DF
Private Key: b'\x0f\xbaE|o1m\x97\x0b\xe5\x18p\x80\xe2\xff\xd1\xc6\xe8\x12\xdc\x88\x9a\x85rW\xd8\x00\xd6\xc36Iv'


2. I sent money from the metamask application using my other ethereum account at 0x1B3b63E30d666CEb1f26933Cb18869aB8e610265

In [15]:
print('Account Balance:', web3.eth.getBalance(account.address))

Account Balance: 100000000000000000


This deploys the contract and sets the sender as the director of the contract

In [26]:
contract = createCoop(web3, 'Psycellium','A Decentralized Cooperatives Network', account)

HASH: b'\xae\xe8\xd2\x97\xdf\x98H\xad\xcfA\xadG\x95Z\xc8\xbd\x9a)\xc5\xa0b6 \x89\xb0rKn\x8b\xee\x94\xf5'
RECIEPT: AttributeDict({'blockHash': HexBytes('0xbcfd1d23939eab0a3aece21775249d2c11feab891567bb11ae133bd47c4c3db9'), 'blockNumber': 4860196, 'contractAddress': '0x9b022f8BcdbC0E9E117640da3E913a9bDE8FF20B', 'cumulativeGasUsed': 2851205, 'from': '0xf3e2db0f694ec09d5546298fa7b801d37a3b45df', 'gasUsed': 1961725, 'logs': [AttributeDict({'address': '0x9b022f8BcdbC0E9E117640da3E913a9bDE8FF20B', 'blockHash': HexBytes('0xbcfd1d23939eab0a3aece21775249d2c11feab891567bb11ae133bd47c4c3db9'), 'blockNumber': 4860196, 'data': '0x', 'logIndex': 0, 'removed': False, 'topics': [HexBytes('0x6206a4cdde152a7caa622013429addc5eeef5054b0b0ce5583913123315aa6af'), HexBytes('0x000000000000000000000000f3e2db0f694ec09d5546298fa7b801d37a3b45df')], 'transactionHash': HexBytes('0xaee8d297df9848adcf41ad47955ac8bd9a29c5a062362089b0724b6e8bee94f5'), 'transactionIndex': 19}), AttributeDict({'address': '0x9b022f8BcdbC0E

In [27]:
print('Cooperative Details')
print('ETH Address:', contract.address)
print('Name:', contract.functions.getCoopName().call())
print('Description:', contract.functions.getCoopDescription().call())
print('Funds:', web3.eth.getBalance(contract.address))
print('Members:', contract.functions.getCoopMembers().call())

Cooperative Details
ETH Address: 0x9b022f8BcdbC0E9E117640da3E913a9bDE8FF20B
Name: Psycellium
Description: A Decentralized Cooperatives Network
Funds: 0
Members: ['0xF3e2DB0F694EC09D5546298fa7B801D37A3b45DF']


# Contract Functions

In [30]:
concise_contract = ConciseContract(contract)
dir(concise_contract)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_classic_contract',
 'address',
 'confirmTransaction',
 'deposit',
 'executeTransaction',
 'factory',
 'getConfirmationCount',
 'getCoopDescription',
 'getCoopMembers',
 'getCoopName',
 'getDirectors',
 'getTransactionCount',
 'isConfirmed',
 'joinCoop',
 'revokeTransaction',
 'setDirector',
 'sumbitRequest',
 'withdraw']

# Creating Accounts and Joining to the Cooperatives

## Account 1

In [31]:
account_1, password_1 = createAccount('account1')

print("Account:", account_1.address)
print("Password:", password_1)
print("Private Key:", account_1.privateKey)

Account: 0x89253625f17B96dfE9fFbF69593B261Fa5ca89aB
Password: account1
Private Key: b"t'\x8e\x89e\x0c\x9d\x11\xe3u^XLC\r\x96rw\xa5\xc4\x97\x92\x8bG\xa9\x19\xd3\x830\x84\xe2\x8f"


In [36]:
print('Account Balance:', web3.eth.getBalance(account_1.address))

Account Balance: 210000000000000000


In [70]:
joinCoop(web3, contract, account_1, 200000000000000000)

1000000
HASH: b'-\x18\x1c\x95\x8c\xcaC`\x08\xb1\xfda\x1f&\xf6\x12\xf0\xae\x05-\xe8\xee\xa3\xaeSz!\xd5M\xefu\xaa'
RECIEPT: AttributeDict({'blockHash': HexBytes('0x1e4168aa2430808d6108b67fc667a5966d2d80200df8f1684724a841e88cc707'), 'blockNumber': 4860360, 'contractAddress': None, 'cumulativeGasUsed': 342204, 'from': '0x89253625f17b96dfe9ffbf69593b261fa5ca89ab', 'gasUsed': 69931, 'logs': [AttributeDict({'address': '0x9b022f8BcdbC0E9E117640da3E913a9bDE8FF20B', 'blockHash': HexBytes('0x1e4168aa2430808d6108b67fc667a5966d2d80200df8f1684724a841e88cc707'), 'blockNumber': 4860360, 'data': '0x', 'logIndex': 1, 'removed': False, 'topics': [HexBytes('0x92fabfa210df88d92c2f5a07ddd43a985b208d8b19b7a78ba1284aa4b287d862'), HexBytes('0x00000000000000000000000089253625f17b96dfe9ffbf69593b261fa5ca89ab')], 'transactionHash': HexBytes('0x2d181c958cca436008b1fd611f26f612f0ae052de8eea3ae537a21d54def75aa'), 'transactionIndex': 5})], 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000

In [71]:
print('Cooperative Details After Account1 Joining')
print('ETH Address:', contract.address)
print('Name:', contract.functions.getCoopName().call())
print('Description:', contract.functions.getCoopDescription().call())
print('Funds:', web3.eth.getBalance(contract.address))
print('Members:', contract.functions.getCoopMembers().call())

Cooperative Details After Account1 Joining
ETH Address: 0x9b022f8BcdbC0E9E117640da3E913a9bDE8FF20B
Name: Psycellium
Description: A Decentralized Cooperatives Network
Funds: 200000000000000000
Members: ['0xF3e2DB0F694EC09D5546298fa7B801D37A3b45DF', '0x89253625f17B96dfE9fFbF69593B261Fa5ca89aB']


## Account 2

In [32]:
account_2, password_2 = createAccount('account2')

print("Account:", account_2.address)
print("Password:", password_2)
print("Private Key:", account_2.privateKey)

Account: 0x5e7375676C1C8A94B957CA370243FEa737247d46
Password: account2
Private Key: b'9\xf0(\xb8&\xa5e\xc0\xff\x8e/f\xa8\xdf\xb2g\x8b\xb2\x84\x80\xa5\xd5\xe7-h\xbc4V5FM\xb8'


In [73]:
print('Account Balance:', web3.eth.getBalance(account_2.address))

Account Balance: 210000000000000000


In [74]:
joinCoop(web3, contract, account_2, 200000000000000000)

1000000
HASH: b'\x86\xa8\x1c\x12\xd0c:\x1b\xa3O\xa3\xe6\xdffZw\xce\xfc\xcbMG\x1b+\xd2\xbfg\x11\xd1\xc2\xd1U\xa5'
RECIEPT: AttributeDict({'blockHash': HexBytes('0x705766ab187ecc2f141db7b5226b62e97c32c74f2c5ec608431fb106d7f640ba'), 'blockNumber': 4860371, 'contractAddress': None, 'cumulativeGasUsed': 1921377, 'from': '0x5e7375676c1c8a94b957ca370243fea737247d46', 'gasUsed': 69931, 'logs': [AttributeDict({'address': '0x9b022f8BcdbC0E9E117640da3E913a9bDE8FF20B', 'blockHash': HexBytes('0x705766ab187ecc2f141db7b5226b62e97c32c74f2c5ec608431fb106d7f640ba'), 'blockNumber': 4860371, 'data': '0x', 'logIndex': 16, 'removed': False, 'topics': [HexBytes('0x92fabfa210df88d92c2f5a07ddd43a985b208d8b19b7a78ba1284aa4b287d862'), HexBytes('0x0000000000000000000000005e7375676c1c8a94b957ca370243fea737247d46')], 'transactionHash': HexBytes('0x86a81c12d0633a1ba34fa3e6df665a77cefccb4d471b2bd2bf6711d1c2d155a5'), 'transactionIndex': 29})], 'logsBloom': HexBytes('0x00000000080000000000000000000000000000000000000000

In [76]:
print('Cooperative Details After Account2 Joining')
print('ETH Address:', contract.address)
print('Name:', contract.functions.getCoopName().call())
print('Description:', contract.functions.getCoopDescription().call())
print('Funds:', web3.eth.getBalance(contract.address))
print('Members:', contract.functions.getCoopMembers().call())

Cooperative Details After Account2 Joining
ETH Address: 0x9b022f8BcdbC0E9E117640da3E913a9bDE8FF20B
Name: Psycellium
Description: A Decentralized Cooperatives Network
Funds: 400000000000000000
Members: ['0xF3e2DB0F694EC09D5546298fa7B801D37A3b45DF', '0x89253625f17B96dfE9fFbF69593B261Fa5ca89aB', '0x5e7375676C1C8A94B957CA370243FEa737247d46']


## Account 3

In [33]:
account_3, password_3 = createAccount('account1')

print("Account:", account_3.address)
print("Password:", password_3)
print("Private Key:", account_3.privateKey)

Account: 0x443b9E6354180f20aE7c453b22f6e8c1BD8cA52d
Password: account1
Private Key: b'\xbd;\x91\xf4\r\xe4pFs\xc5#S\xd0$\xceh&E\xe9\xccT\x0e\xf8v7\xba\x11\xf7=m\t\x05'


In [80]:
print('Account Balance:', web3.eth.getBalance(account_3.address))

Account Balance: 210000000000000000


In [81]:
joinCoop(web3, contract, account_3, 200000000000000000)

1000000
HASH: b"\x1f-\x19\xa8cV\xfc@\xb4\xb2|\xcb\xca\xd6i\x98\xd08/\xb3\xa5\xf6\xa0'\xf6;\xfb>%\x00m\xc4"
RECIEPT: AttributeDict({'blockHash': HexBytes('0xe1cf4562b0135f915aef634200683178d3d98590fed6c331c0723ee6d82905ef'), 'blockNumber': 4860389, 'contractAddress': None, 'cumulativeGasUsed': 1180749, 'from': '0x443b9e6354180f20ae7c453b22f6e8c1bd8ca52d', 'gasUsed': 69931, 'logs': [AttributeDict({'address': '0x9b022f8BcdbC0E9E117640da3E913a9bDE8FF20B', 'blockHash': HexBytes('0xe1cf4562b0135f915aef634200683178d3d98590fed6c331c0723ee6d82905ef'), 'blockNumber': 4860389, 'data': '0x', 'logIndex': 7, 'removed': False, 'topics': [HexBytes('0x92fabfa210df88d92c2f5a07ddd43a985b208d8b19b7a78ba1284aa4b287d862'), HexBytes('0x000000000000000000000000443b9e6354180f20ae7c453b22f6e8c1bd8ca52d')], 'transactionHash': HexBytes('0x1f2d19a86356fc40b4b27ccbcad66998d0382fb3a5f6a027f63bfb3e25006dc4'), 'transactionIndex': 34})], 'logsBloom': HexBytes('0x000000000000000000000000000000000000000000000000000000000

In [82]:
print('Cooperative Details After Account3 Joining')
print('ETH Address:', contract.address)
print('Name:', contract.functions.getCoopName().call())
print('Description:', contract.functions.getCoopDescription().call())
print('Funds:', web3.eth.getBalance(contract.address))
print('Members:', contract.functions.getCoopMembers().call())

Cooperative Details After Account3 Joining
ETH Address: 0x9b022f8BcdbC0E9E117640da3E913a9bDE8FF20B
Name: Psycellium
Description: A Decentralized Cooperatives Network
Funds: 600000000000000000
Members: ['0xF3e2DB0F694EC09D5546298fa7B801D37A3b45DF', '0x89253625f17B96dfE9fFbF69593B261Fa5ca89aB', '0x5e7375676C1C8A94B957CA370243FEa737247d46', '0x443b9E6354180f20aE7c453b22f6e8c1BD8cA52d']


# Set New Director

In [84]:
setDirector(web3, contract, account, account_1)

HASH: b'\xefr\xae\xb6\xbb\x10\x11\xd4\x86G-S?iF\xbb\xeb\xa1\x0b\xf6}mp\x0f\xe8D7\xc3:\xb5\x15S'
RECIEPT: AttributeDict({'blockHash': HexBytes('0xeb95ef7771ed4739b6577abac5b81f64958347983e3a5d7c76d2db8f9e3c2023'), 'blockNumber': 4860405, 'contractAddress': None, 'cumulativeGasUsed': 6131718, 'from': '0xf3e2db0f694ec09d5546298fa7b801d37a3b45df', 'gasUsed': 70681, 'logs': [AttributeDict({'address': '0x9b022f8BcdbC0E9E117640da3E913a9bDE8FF20B', 'blockHash': HexBytes('0xeb95ef7771ed4739b6577abac5b81f64958347983e3a5d7c76d2db8f9e3c2023'), 'blockNumber': 4860405, 'data': '0x', 'logIndex': 85, 'removed': False, 'topics': [HexBytes('0x6206a4cdde152a7caa622013429addc5eeef5054b0b0ce5583913123315aa6af'), HexBytes('0x00000000000000000000000089253625f17b96dfe9ffbf69593b261fa5ca89ab')], 'transactionHash': HexBytes('0xef72aeb6bb1011d486472d533f6946bbeba10bf67d6d700fe84437c33ab51553'), 'transactionIndex': 102})], 'logsBloom': HexBytes('0x000000000000000000000000000000000000000000000000000000000100000000

In [85]:
print('Directors:', contract.functions.getDirectors().call())

Directors: ['0xF3e2DB0F694EC09D5546298fa7B801D37A3b45DF', '0x89253625f17B96dfE9fFbF69593B261Fa5ca89aB']


# Submit Request

In [138]:
submitRequest(web3, contract, account, account_3, 200000000000000000)

HASH: b'\x00#UB\xfdb\x02\x9c;\xb8\x98\xddM7\x99\xdd\x13\xe0}\xaa\xc6\xcb\x1c\x14\x11\xedglW6\x9b\x16'
RECIEPT: AttributeDict({'blockHash': HexBytes('0xb4233656cc6d78264456f050d7e1e2090433e0b9d8847e2e56885e961bb0f00f'), 'blockNumber': 4860667, 'contractAddress': None, 'cumulativeGasUsed': 5411822, 'from': '0xf3e2db0f694ec09d5546298fa7b801d37a3b45df', 'gasUsed': 162691, 'logs': [AttributeDict({'address': '0x9b022f8BcdbC0E9E117640da3E913a9bDE8FF20B', 'blockHash': HexBytes('0xb4233656cc6d78264456f050d7e1e2090433e0b9d8847e2e56885e961bb0f00f'), 'blockNumber': 4860667, 'data': '0x', 'logIndex': 4, 'removed': False, 'topics': [HexBytes('0xc0ba8fe4b176c1714197d43b9cc6bcf797a4a7461c5fe8d0ef6e184ae7601e51'), HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000')], 'transactionHash': HexBytes('0x00235542fd62029c3bb898dd4d3799dd13e07daac6cb1c1411ed676c57369b16'), 'transactionIndex': 24}), AttributeDict({'address': '0x9b022f8BcdbC0E9E117640da3E913a9bDE8FF20B', 'blockHash': He

In [139]:
print('Account Balance:', web3.eth.getBalance(account_3.address))

Account Balance: 209388799000000000


In [140]:
print('Funds:', web3.eth.getBalance(contract.address))

Funds: 400000000000000000
